In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_100479/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_41_8,0.996444,0.612037,0.532101,0.993538,0.001272,0.230312,0.327773,0.002376,0.016023,0.035667,0.991629,0.035984,143.334120,321.187019,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_41_7,0.996438,0.612136,0.532363,0.993579,0.001274,0.230253,0.327590,0.002361,0.016039,0.035698,0.991614,0.036015,143.330636,321.183535,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_41_6,0.996432,0.612051,0.531844,0.993623,0.001277,0.230303,0.327953,0.002345,0.016067,0.035730,0.991600,0.036047,143.327113,321.180012,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_41_9,0.996422,0.612037,0.532696,0.993271,0.001280,0.230312,0.327356,0.002474,0.015957,0.035779,0.991576,0.036098,143.321523,321.174422,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_44_9,0.996416,0.620960,0.552970,0.994951,0.001282,0.225015,0.313154,0.001868,0.016915,0.035810,0.991562,0.036128,143.318125,321.171025,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_4,0.367047,0.239913,0.159420,0.388998,0.226446,0.451220,0.588844,0.187136,0.237121,0.475863,-0.490077,0.480093,132.970501,310.823400,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.359401,0.242333,0.162812,0.388923,0.229181,0.449784,0.586468,0.187159,0.238919,0.478729,-0.508078,0.482984,132.946485,310.799384,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.223963,0.151350,0.088268,0.217422,0.277636,0.503795,0.638687,0.239686,0.262965,0.526911,-0.826921,0.531595,132.562892,310.415791,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.205692,0.126707,0.033812,0.235684,0.284172,0.518424,0.676835,0.234093,0.280889,0.533078,-0.869933,0.537816,132.516352,310.369251,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
